In [1]:
import sys
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive/Colab_Notebooks/DeepSynergy"
import os
os.chdir(path)

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import pickle 
import gzip

In [3]:
#contains the data in both feature ordering ways (drug A - drug B - cell line and drug B - drug A - cell line)
#in the first half of the data the features are ordered (drug A - drug B - cell line)
#in the second half of the data the features are ordered (drug B - drug A - cell line)
file = gzip.open('./dataset/X_15_emb_c30.p', 'rb')
X_15 = pickle.load(file)
file.close()

In [4]:
X_15.shape

(6006, 50)

In [5]:
#contains synergy values and fold split (numbers 0-4)   
labels_15 = pd.read_csv('dataset/labels_15.csv', index_col=0) 
#labels are duplicated for the two different ways of ordering in the data
labels_15 = pd.concat([labels_15, labels_15])

In [6]:
# fold 0 is used for testing and fold 1 for validation (hyperparamter selection)
test_fold = 0
val_fold = 1

#indices of training data for hyperparameter selection: fold 2, 3, 4
idx_tr = np.where(np.logical_and(labels_15['fold']!=test_fold, labels_15['fold']!=val_fold))
#indices of validation data for hyperparameter selection: fold 1
idx_val = np.where(labels_15['fold']==val_fold)

#indices of training data for model testing: fold 1, 2, 3, 4
idx_train = np.where(labels_15['fold']!=test_fold)
#indices of test data for model testing: fold 0
idx_test = np.where(labels_15['fold']==test_fold)

In [7]:
X_tr = X_15[idx_tr]
X_val = X_15[idx_val]
X_train = X_15[idx_train]
X_test = X_15[idx_test]

y_tr = labels_15.iloc[idx_tr]['synergy'].values
y_val = labels_15.iloc[idx_val]['synergy'].values
y_train = labels_15.iloc[idx_train]['synergy'].values
y_test = labels_15.iloc[idx_test]['synergy'].values

In [8]:
with gzip.open('./dataset/data_15_emb_c30_testfold%d.p'%test_fold, "wb") as f:
    pickle.dump((X_tr, X_val, X_train, X_test, y_tr, y_val, y_train, y_test), f)